In [1]:
from fastai.text import *
from sklearn.model_selection import train_test_split

In [2]:
# run this to see what has already been imported
#whos

In [3]:
# pandas doesn't understand ~, so provide full path
base_path = Path('/home/jupyter/mimic')

```python
# run this the first time to covert CSV to Pickle file
df = pd.read_csv(base_path/'NOTEEVENTS.csv', low_memory=False, memory_map=True)
df.to_pickle(base_path/'noteevents.pickle')
```

In [4]:
# this is much faster than reading a csv
df = pd.read_pickle(base_path/'noteevents.pickle')

In [5]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [6]:
df.dtypes

ROW_ID           int64
SUBJECT_ID       int64
HADM_ID        float64
CHARTDATE       object
CHARTTIME       object
STORETIME       object
CATEGORY        object
DESCRIPTION     object
CGID           float64
ISERROR        float64
TEXT            object
dtype: object

In [7]:
df.shape

(2083180, 11)

In [8]:
# split data into train and test sets
seed = 42
test_size = 0.333333333
train, test = train_test_split(df, test_size=test_size, random_state=seed)

In [9]:
train.shape

(1388786, 11)

In [10]:
test.shape

(694394, 11)

In [11]:
#data = TextClasDataBunch.from_df(base_path, train_df = train, valid_df = test, text_cols='TEXT')
#data.show_batch()

data = (TextList.from_df(df, 'texts.csv', cols='TEXT')
                .split_by_rand_pct(valid_pct=0.2, seed=seed) 
                .label_from_df(cols='CATEGORY')
                .databunch())

In [12]:
bs=48
data_lm = (TextList.from_df(df, 'texts.csv', cols='TEXT')
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
data_lm.save('mimic_lm.pickle')

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
data

In [ ]:
## why does this only seem to use CPU?
# both textclasdatabunch and textlist...
# run out of memory at 32 GB, error at 52 GB, trying 72GB now... got down to only 440MB free; if crash again, increase memory